Path

In [ ]:
!pip install torch_geometric category_encoders

from google.colab import drive
import os
import warnings

warnings.simplefilter(action='ignore', category=FutureWarning)

drive.mount("/content/drive/")
os.chdir("/content/drive/MyDrive/BikeCSDI/")

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


Import & Config

In [ ]:
import torch
from torch import Tensor
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from torch_geometric.nn import GCNConv, global_mean_pool
from torch_geometric.data import Data, Batch
from torch.optim import Adam
import category_encoders as ce
from tqdm import tqdm
import math
import yaml
import re
import numpy as np
import typing as ty
import torch.nn.init as nn_init
import pickle
import pandas as pd
import matplotlib.pyplot as plt

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

bike_components = {
    "Seat Tube": ["Seat tube length", "Seat tube extension2", "Seat tube diameter", "Seatpost setback", "Seatpost LENGTH", "Stack", "Lower stack height", "Upper stack height"],
    "Head Tube": ["Head tube upper extension2", "Head angle", "Head tube lower extension2", "Head tube diameter"],
    "Top Tube": ["Top tube rear diameter", "Top tube front diameter"],
    "Down Tube": ["Down tube front diameter", "Down tube rear diameter"],
    "Chain Stay": ["CHAINSTAYAUXrearDIAMETER", "Chain stay horizontal diameter", "Chain stay position on BB", "Chain stay taper", "Chain stay back diameter", "Chain stay vertical diameter"],
    "Seat Stay": ["Seat stay junction0", "Seat stay bottom diameter", "SEATSTAY_HF", "SEATSTAY_HR", "SEATSTAYTAPERLENGTH"],
    "Fork": ["FORK0R", "FORK0L"],
    "Saddle": ["Saddle P", "Saddle thickness", "Saddle angle", "Saddle J", "Saddle H", "Saddle E", "SADDLETIPtoMIDDLE", "Saddle length"],
    "Wheel": ["Wheel width rear", "Wheel width front", "Wheel diameter front", "Wheel diameter rear", "SPOKES composite front", "SPOKES front", "SPOKES rear", "SPOKES composite rear", "ERD rear", "ERD front"],
    "Handle": ["Road bar reach", "Road bar drop", "Brake lever position", "Bullhorn angle", "HBAREXTEND", "Handlebar angle", "MtnBar angle", "HBARTHETA", "Pedal width", "Stem angle", "Stem length"],
    "BB": ["BB textfield", "BB length", "BB diameter"],
    }

bike_edge = torch.tensor([[0, 0, 0, 1, 1, 1, 1, 3, 3, 4, 4, 4, 5, 6], [2, 5, 7, 2, 3, 6, 9, 6, 10, 5, 8, 10, 8, 8]])

config = {
    "train": {
        "epochs": 10,
        "batch_size": 400,
        "lr": 4e-4,
    },
    "diffusion": {
        "layers": 4,
        "channels": 128,
        "nheads": 4,
        "diffusion_embedding_dim": 128,
        "beta_start": 0.0001,
        "beta_end": 0.5,
        "num_steps": 100,
        "schedule": "quad",
        "mixed": True,
        "token_emb_dim": 8,
    },
    "model": {
        "is_unconditional": 0,
        "feature_emb": 8,
        "time_emb": 128,
        "target_strategy": "random",
        "mixed": True,
        "token_emb_dim": 8,
        "graph_cond_dim": 32,
        "test_missing_ratio": 0.1,
        "graph_input_dim": 16,
        "graph_hidden_dim": 32,
        "graph_output_dim": 32,
    },
  }

seed = 4096

nfold = 100

nsample = 5

masked_features = []

timepoints = 182

torch.manual_seed(seed)
np.random.seed(seed)
torch.cuda.manual_seed(seed)

GraphCSDI

In [ ]:
def get_torch_trans(heads=8, layers=1, channels=64):
    encoder_layer = nn.TransformerEncoderLayer(
        d_model=channels, nhead=heads, dim_feedforward=64, activation="gelu"
    )
    return nn.TransformerEncoder(encoder_layer, num_layers=layers)


def Conv1d_with_init(in_channels, out_channels, kernel_size):
    layer = nn.Conv1d(in_channels, out_channels, kernel_size)
    # Weight initialization
    nn.init.kaiming_normal_(layer.weight)
    return layer


class DiffusionEmbedding(nn.Module):
    def __init__(self, num_steps, embedding_dim=128, projection_dim=None):
        super().__init__()
        if projection_dim is None:
            projection_dim = embedding_dim
        self.register_buffer(
            "embedding",
            self._build_embedding(num_steps, embedding_dim / 2),
            persistent=False,
        )
        self.projection1 = nn.Linear(embedding_dim, projection_dim)
        self.projection2 = nn.Linear(projection_dim, projection_dim)

    def forward(self, diffusion_step):
        x = self.embedding[diffusion_step]
        x = self.projection1(x)
        x = F.silu(x)
        x = self.projection2(x)
        x = F.silu(x)
        return x

    # t_embedding(t). The embedding dimension is 128 in total for every time step t.
    def _build_embedding(self, num_steps, dim=64):
        steps = torch.arange(num_steps).unsqueeze(1)  # (T,1)
        frequencies = 10.0 ** (torch.arange(dim) / (dim - 1) * 4.0).unsqueeze(
            0
        )  # (1,dim)
        table = steps * frequencies  # (T,dim)
        table = torch.cat([torch.sin(table), torch.cos(table)], dim=1)  # (T,dim*2)
        return table


class diff_CSDI(nn.Module):
    def __init__(self, config, inputdim=2):
        super().__init__()
        self.config = config
        self.channels = config["channels"]

        self.diffusion_embedding = DiffusionEmbedding(
            num_steps=config["num_steps"],
            embedding_dim=config["diffusion_embedding_dim"],
        )

        self.token_emb_dim = config["token_emb_dim"] if config["mixed"] else 1
        inputdim = 2 * self.token_emb_dim

        self.input_projection = Conv1d_with_init(inputdim, self.channels, 1)
        self.output_projection1 = Conv1d_with_init(self.channels, self.channels, 1)
        self.output_projection2 = Conv1d_with_init(self.channels, self.token_emb_dim, 1)
        nn.init.zeros_(self.output_projection2.weight)

        self.residual_layers = nn.ModuleList(
            [
                ResidualBlock(
                    side_dim=config["side_dim"],
                    channels=self.channels,
                    diffusion_embedding_dim=config["diffusion_embedding_dim"],
                    nheads=config["nheads"],
                )
                for _ in range(config["layers"])
            ]
        )

    def forward(self, x, cond_info, diffusion_step):
        B, inputdim, K, L = x.shape

        x = x.reshape(B, inputdim, K * L)
        x = self.input_projection(x)
        x = F.relu(x)
        x = x.reshape(B, self.channels, K, L)

        diffusion_emb = self.diffusion_embedding(diffusion_step)

        skip = []
        for layer in self.residual_layers:
            x, skip_connection = layer(x, cond_info, diffusion_emb)
            skip.append(skip_connection)

        x = torch.sum(torch.stack(skip), dim=0) / math.sqrt(len(self.residual_layers))
        x = x.reshape(B, self.channels, K * L)

        x = self.output_projection1(x)
        x = F.relu(x)
        x = self.output_projection2(x)
        if self.config["mixed"]:
            x = x.permute(0, 2, 1)
            x = x.reshape(B, K, L * self.token_emb_dim)
        else:
            x = x.reshape(B, K, L)
        return x


class ResidualBlock(nn.Module):
    def __init__(self, side_dim, channels, diffusion_embedding_dim, nheads):
        super().__init__()
        self.diffusion_projection = nn.Linear(diffusion_embedding_dim, channels)
        self.cond_projection = Conv1d_with_init(side_dim, 2 * channels, 1)
        self.mid_projection = Conv1d_with_init(channels, 2 * channels, 1)
        self.output_projection = Conv1d_with_init(channels, 2 * channels, 1)

        # Temporal Transformer layer
        self.time_layer = get_torch_trans(heads=nheads, layers=1, channels=channels)
        # Feature Transformer layer
        self.feature_layer = get_torch_trans(heads=nheads, layers=1, channels=channels)

    def forward_time(self, y, base_shape):
        B, channel, K, L = base_shape
        if L == 1:
            return y
        y = y.reshape(B, channel, K, L).permute(0, 2, 1, 3).reshape(B * K, channel, L)
        # (B*K, C, L) -> (L, B*K, C) -> (B*K, C, L)
        # input shape for transformerencoder: [seq, batch, emb]
        y = self.time_layer(y.permute(2, 0, 1)).permute(1, 2, 0)
        y = y.reshape(B, K, channel, L).permute(0, 2, 1, 3).reshape(B, channel, K * L)
        return y

    def forward_feature(self, y, base_shape):
        B, channel, K, L = base_shape
        if K == 1:
            return y
        y = y.reshape(B, channel, K, L).permute(0, 3, 1, 2).reshape(B * L, channel, K)
        # (B*L, C, K) -> (K, B*L, C) -> (B*L, C, K)
        y = self.feature_layer(y.permute(2, 0, 1)).permute(1, 2, 0)
        y = y.reshape(B, L, channel, K).permute(0, 2, 3, 1).reshape(B, channel, K * L)
        return y

    def forward(self, x, cond_info, diffusion_emb):
        B, channel, K, L = x.shape
        base_shape = x.shape
        x = x.reshape(B, channel, K * L)

        # diffusion_emb is
        diffusion_emb = self.diffusion_projection(diffusion_emb).unsqueeze(
            -1
        )  # (B,channel,1)
        y = x + diffusion_emb

        y = self.forward_time(y, base_shape)
        y = self.forward_feature(y, base_shape)  # (B,channel,K*L)
        y = self.mid_projection(y)  # (B,2*channel,K*L)

        _, cond_dim, _, _ = cond_info.shape
        cond_info = cond_info.reshape(B, cond_dim, K * L)
        cond_info = self.cond_projection(cond_info)  # (B,2*channel,K*L)
        y = y + cond_info

        gate, filter = torch.chunk(y, 2, dim=1)
        y = torch.sigmoid(gate) * torch.tanh(filter)  # (B,channel,K*L)
        y = self.output_projection(y)

        residual, skip = torch.chunk(y, 2, dim=1)
        x = x.reshape(base_shape)
        residual = residual.reshape(base_shape)
        skip = skip.reshape(base_shape)
        return (x + residual) / math.sqrt(2.0), skip


class Tokenizer(nn.Module):
    def __init__(
        self,
        d_numerical: int,
        categories: ty.Optional[ty.List[int]],
        d_token: int,
        bias: bool,
    ) -> None:
        super().__init__()

        d_bias = d_numerical + len(categories)
        category_offsets = torch.tensor([0] + categories[:-1]).cumsum(0)
        self.d_token = d_token
        self.register_buffer("category_offsets", category_offsets)
        self.category_embeddings = nn.Embedding(sum(categories) + 1, self.d_token)
        self.category_embeddings.weight.requires_grad = False
        nn_init.kaiming_uniform_(self.category_embeddings.weight, a=math.sqrt(5))

        self.weight = nn.Parameter(Tensor(d_numerical, self.d_token))
        self.weight.requires_grad = False

        self.bias = nn.Parameter(Tensor(d_bias, self.d_token)) if bias else None
        nn_init.kaiming_uniform_(self.weight, a=math.sqrt(5))
        if self.bias is not None:
            nn_init.kaiming_uniform_(self.bias, a=math.sqrt(5))
            self.bias.requires_grad = False

    @property
    def n_tokens(self) -> int:
        return len(self.weight) + (
            0 if self.category_offsets is None else len(self.category_offsets)
        )

    def forward(self, x_num: Tensor, x_cat: ty.Optional[Tensor]) -> Tensor:
        x_some = x_num if x_cat is None else x_cat
        x_cat = x_cat.type(torch.int32)

        assert x_some is not None
        x = self.weight.T * x_num

        if x_cat is not None:
            x = x[:, np.newaxis, :, :]
            x = x.permute(0, 1, 3, 2)
            x = torch.cat(
                [x, self.category_embeddings(x_cat + self.category_offsets[None])],
                dim=2,
            )
        if self.bias is not None:
            x = x + self.bias[None]

        return x

    def recover(self, Batch, d_numerical):
        B, L, K = Batch.shape
        L_new = int(L / self.d_token)
        Batch = Batch.reshape(B, L_new, self.d_token)
        Batch = Batch - self.bias

        Batch_numerical = Batch[:, :d_numerical, :]
        Batch_numerical = Batch_numerical / self.weight
        Batch_numerical = torch.mean(Batch_numerical, 2, keepdim=False)

        Batch_cat = Batch[:, d_numerical:, :]
        new_Batch_cat = torch.zeros([Batch_cat.shape[0], Batch_cat.shape[1]])
        for i in range(Batch_cat.shape[1]):
            token_start = self.category_offsets[i] + 1
            if i == Batch_cat.shape[1] - 1:
                token_end = self.category_embeddings.weight.shape[0] - 1
            else:
                token_end = self.category_offsets[i + 1]
            emb_vec = self.category_embeddings.weight[token_start : token_end + 1, :]
            for j in range(Batch_cat.shape[0]):
                distance = torch.norm(emb_vec - Batch_cat[j, i, :], dim=1)
                nearest = torch.argmin(distance)
                new_Batch_cat[j, i] = nearest + 1
            new_Batch_cat = new_Batch_cat.to(Batch_numerical.device)
        return torch.cat([Batch_numerical, new_Batch_cat], dim=1)


class GNNModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super().__init__()
        self.conv1 = GCNConv(input_dim, hidden_dim)
        self.conv2 = GCNConv(hidden_dim, output_dim)
        self.output_proj = nn.Linear(output_dim, output_dim)
        self.bn = nn.BatchNorm1d(output_dim)

    def forward(self, data):
        x, edge_index, batch = data.x, data.edge_index, data.batch
        x = F.relu(self.conv1(x, edge_index))
        x = F.relu(self.conv2(x, edge_index))
        x = global_mean_pool(x, batch)
        x = self.bn(F.relu(self.output_proj(x)))
        return x

class StructuralEmbedding(nn.Module):
  def __init__(self, input_dim, output_dim=16):
    super().__init__()
    self.proj = nn.Linear(input_dim, output_dim)
    self.bn = nn.BatchNorm1d(output_dim)

  def forward(self, x):
    x = F.relu(self.proj(x))
    x = self.bn(x)
    return x


class GraphEmbedding(nn.Module):
  def __init__(self, config, device):
    super().__init__()
    graph_input_dim, graph_hidden_dim, graph_output_dim = config["model"]["graph_input_dim"], config["model"]["graph_hidden_dim"], config["model"]["graph_output_dim"]
    self.proj = nn.ModuleDict({
        component: StructuralEmbedding(len(bike_components[component]), graph_input_dim) for component in bike_components
    })
    self.gnn = GNNModel(graph_input_dim, graph_hidden_dim, graph_output_dim)
    self.device = device

  def forward(self, batch):
    graph_inputs = [self.proj[component](batch[component].to(self.device)) for component in bike_components]
    graphs = [Data(x=torch.stack([graph_inputs[n][i] for n in range(len(graph_inputs))]), edge_index=bike_edge) for i in range(len(graph_inputs[0]))]
    graph_batch = Batch.from_data_list(graphs).to(self.device)
    graph_cond = self.gnn(graph_batch)  # (B, G)
    return graph_cond


class CSDI_base(nn.Module):
    def __init__(self, target_dim, config, device):
        super().__init__()
        self.device = device
        self.target_dim = target_dim

        # load embedding vector dimension.
        self.emb_feature_dim = config["model"]["feature_emb"]
        self.emb_time_dim = config["model"]["time_emb"]
        self.graph_cond_dim = config["model"]["graph_cond_dim"]

        self.emb_total_dim = self.emb_feature_dim + self.emb_time_dim + self.graph_cond_dim
        self.graph_embed = GraphEmbedding(config, device)

        self.is_unconditional = config["model"]["is_unconditional"]
        self.target_strategy = config["model"]["target_strategy"]

        # For categorical variables
        self.mixed = config["model"]["mixed"]

        with open("./data/transformed_columns.pk", "rb") as f:
            cont_list, num_cate_list = pickle.load(f)

        self.cont_list = cont_list
        self.num_cate_list = num_cate_list

        if self.mixed:
            self.token_dim = config["model"]["token_emb_dim"]

            # set tokenizer
            d_numerical = len(self.cont_list)
            categories = self.num_cate_list
            d_token = self.token_dim
            token_bias = True

            self.tokenizer = Tokenizer(d_numerical, categories, d_token, token_bias)

        if self.is_unconditional == False:
            self.emb_total_dim += 1  # for conditional mask

        self.embed_layer = nn.Embedding(
            num_embeddings=self.target_dim, embedding_dim=self.emb_feature_dim
        )

        config_diff = config["diffusion"]
        config_diff["side_dim"] = self.emb_total_dim

        input_dim = 1 if self.is_unconditional == True else 2
        tot_feature_num = len(cont_list) + len(num_cate_list)
        self.diffmodel = diff_CSDI(config_diff, input_dim)

        # parameters for diffusion models
        self.num_steps = config_diff["num_steps"]
        if config_diff["schedule"] == "quad":
            self.beta = (
                np.linspace(
                    config_diff["beta_start"] ** 0.5,
                    config_diff["beta_end"] ** 0.5,
                    self.num_steps,
                )
                ** 2
            )
        elif config_diff["schedule"] == "linear":
            self.beta = np.linspace(
                config_diff["beta_start"], config_diff["beta_end"], self.num_steps
            )

        self.alpha_hat = 1 - self.beta
        self.alpha = np.cumprod(self.alpha_hat)
        self.alpha_torch = (
            torch.tensor(self.alpha).float().to(self.device).unsqueeze(1).unsqueeze(1)
        )

    def get_randmask(self, observed_mask):
        rand_for_mask = torch.rand_like(observed_mask) * observed_mask
        rand_for_mask = rand_for_mask.reshape(len(rand_for_mask), -1)

        for i in range(len(observed_mask)):
            sample_ratio = np.random.rand()
            num_observed = observed_mask[i].sum().item()
            num_masked = round(num_observed * sample_ratio)
            rand_for_mask[i][rand_for_mask[i].topk(num_masked).indices] = -1
        cond_mask = (rand_for_mask > 0).reshape(observed_mask.shape).float()
        return cond_mask

    def time_embedding(self, pos, d_model=128):
        pe = torch.zeros(pos.shape[0], pos.shape[1], d_model).to(self.device)
        position = pos.unsqueeze(2)
        div_term = 1 / torch.pow(
            10000.0, torch.arange(0, d_model, 2).to(self.device) / d_model
        )
        pe[:, :, 0::2] = torch.sin(position * div_term)
        pe[:, :, 1::2] = torch.cos(position * div_term)
        return pe

    def get_side_info(self, graph_features, observed_tp, cond_mask):
        B, K, L = cond_mask.shape

        graph_embed = self.graph_embed(graph_features)  # (B,Graph_emb)
        graph_embed = graph_embed.unsqueeze(1).unsqueeze(1).expand(-1, L, K, -1)

        time_embed = self.time_embedding(observed_tp, self.emb_time_dim)  # (B,L,emb)
        time_embed = time_embed.unsqueeze(2).expand(-1, -1, K, -1)
        feature_embed = self.embed_layer(
            torch.arange(self.target_dim).to(self.device)
        )  # (K,emb)
        feature_embed = feature_embed.unsqueeze(0).unsqueeze(0).expand(B, L, -1, -1)
        side_info = torch.cat([graph_embed, time_embed, feature_embed], dim=-1)  # (B,L,K,*)
        side_info = side_info.permute(0, 3, 2, 1)  # (B,*,K,L)

        if self.is_unconditional == False:
            side_mask = cond_mask.unsqueeze(1)  # (B,1,K,L)
            side_info = torch.cat([side_info, side_mask], dim=1)

        return side_info

    def calc_loss_valid(
        self, observed_data, cond_mask, observed_mask, side_info, is_train
    ):
        loss_sum = 0
        for t in range(self.num_steps):
            loss = self.calc_loss(
                observed_data, cond_mask, observed_mask, side_info, is_train, set_t=t
            )
            loss_sum += loss.detach()
        return loss_sum / self.num_steps

    def calc_loss(
        self, observed_data, cond_mask, observed_mask, side_info, is_train, set_t=-1
    ):
        B, K, L = observed_data.shape
        if is_train != 1:
            t = (torch.ones(B) * set_t).long().to(self.device)
        else:
            t = torch.randint(0, self.num_steps, [B]).to(self.device)
        current_alpha = self.alpha_torch[t]  # (B,1,1)
        noise = torch.randn_like(observed_data)
        # Perform forward step. Adding noise to all data.
        noisy_data = (current_alpha**0.5) * observed_data + (
            1.0 - current_alpha
        ) ** 0.5 * noise
        total_input = self.set_input_to_diffmodel(noisy_data, observed_data, cond_mask)
        predicted = self.diffmodel(total_input, side_info, t)  # (B,K,L*token_dim)

        target_mask = observed_mask - cond_mask
        target_mask = torch.repeat_interleave(target_mask, self.token_dim, dim=2)
        residual = (noise - predicted) * target_mask
        num_eval = target_mask.sum()
        loss = (residual**2).sum() / (num_eval if num_eval > 0 else 1)
        return loss

    def set_input_to_diffmodel(self, noisy_data, observed_data, cond_mask):

        cond_mask = torch.repeat_interleave(cond_mask, self.token_dim, dim=2)

        cond_obs = (cond_mask * observed_data).unsqueeze(1)
        noisy_target = ((1 - cond_mask) * noisy_data).unsqueeze(1)
        total_input = torch.cat([cond_obs, noisy_target], dim=1)  # (B,2,K,L)
        B, old_input_dim, K, L = total_input.shape
        total_input = total_input.reshape(
            B, old_input_dim, K, int(L / self.token_dim), self.token_dim
        )
        total_input = total_input.permute(0, 1, 4, 2, 3)
        total_input = total_input.reshape(
            B, old_input_dim * self.token_dim, K, int(L / self.token_dim)
        )

        return total_input

    def impute(self, observed_data, cond_mask, side_info, n_samples):

        B, K, L = observed_data.shape
        cond_mask = torch.repeat_interleave(cond_mask, self.token_dim, dim=2)

        imputed_samples = torch.zeros(B, n_samples, K, L).to(self.device)
        # Perform n_samples times of forward and backward pass for same input data.
        for i in range(n_samples):
            if self.is_unconditional == True:
                noisy_obs = observed_data
                noisy_cond_history = []
                # perform T steps forward
                for t in range(self.num_steps):
                    noise = torch.randn_like(noisy_obs)
                    noisy_obs = (self.alpha_hat[t] ** 0.5) * noisy_obs + self.beta[
                        t
                    ] ** 0.5 * noise
                    noisy_cond_history.append(noisy_obs * cond_mask)

            current_sample = torch.randn_like(observed_data)
            # perform T steps backward
            for t in range(self.num_steps - 1, -1, -1):
                if self.is_unconditional == True:
                    diff_input = (
                        cond_mask * noisy_cond_history[t]
                        + (1.0 - cond_mask) * current_sample
                    )
                    diff_input = diff_input.unsqueeze(1)  # (B,1,K,L)
                else:
                    # fix original x^{co} as condition
                    cond_obs = (cond_mask * observed_data).unsqueeze(1)
                    noisy_target = ((1 - cond_mask) * current_sample).unsqueeze(1)
                    diff_input = torch.cat([cond_obs, noisy_target], dim=1)  # (B,2,K,L)
                    B, old_input_dim, K, L = diff_input.shape
                    diff_input = diff_input.reshape(
                        B, old_input_dim, K, int(L / self.token_dim), self.token_dim
                    )
                    diff_input = diff_input.permute(0, 1, 4, 2, 3)
                    diff_input = diff_input.reshape(
                        B, old_input_dim * self.token_dim, K, int(L / self.token_dim)
                    )

                predicted = self.diffmodel(
                    diff_input, side_info, torch.tensor([t]).to(self.device)
                )  # (B,K,L)
                coeff1 = 1 / self.alpha_hat[t] ** 0.5
                coeff2 = (1 - self.alpha_hat[t]) / (1 - self.alpha[t]) ** 0.5

                current_sample = coeff1 * (current_sample - coeff2 * predicted)

                if t > 0:
                    noise = torch.randn_like(current_sample)
                    sigma = (
                        (1.0 - self.alpha[t - 1]) / (1.0 - self.alpha[t]) * self.beta[t]
                    ) ** 0.5
                    current_sample += sigma * noise
            imputed_samples[:, i] = current_sample.detach()

        return imputed_samples

    def forward(self, batch, is_train=1):
        (
            observed_data,
            observed_mask,
            observed_tp,
            gt_mask,
            for_pattern_mask,
            _,
            graph_features,
        ) = self.process_data(batch)

        # In testing, using `gt_mask` (generated with fixed missing rate) as cond_mask.
        # In training, generate random mask as cond_mask
        if is_train == 0:
            cond_mask = gt_mask
        else:
            cond_mask = self.get_randmask(observed_mask)

        cond_mask[:, :, :self.graph_cond_dim] = True

        side_info = self.get_side_info(graph_features, observed_tp, cond_mask)

        # The main calculation procedures are in `self.calc_loss()`
        loss_func = self.calc_loss if is_train == 1 else self.calc_loss_valid
        return loss_func(observed_data, cond_mask, observed_mask, side_info, is_train)

    def evaluate(self, batch, n_samples):
        (
            observed_data,
            observed_mask,
            observed_tp,
            gt_mask,
            _,
            cut_length,
            graph_features,
        ) = self.process_data(batch)

        with torch.no_grad():
            # gt_mask: 0 for missing elements and manully maksed elements
            cond_mask = gt_mask
            # target_mask: 1 for manually masked elements
            target_mask = observed_mask - cond_mask
            side_info = self.get_side_info(graph_features, observed_tp, cond_mask)
            samples = self.impute(observed_data, cond_mask, side_info, n_samples)

        return samples, observed_data, target_mask, observed_mask


class BikeGraphCSDI(CSDI_base):
    def __init__(self, config, device, target_dim=1):
        super().__init__(target_dim, config, device)

    def process_data(self, batch):
        # Insert K=1 axis. All mask now with shape (B, 1, L). L=# of attributes.
        observed_data = batch["observed_data"][:, np.newaxis, :]
        observed_data = observed_data.to(self.device).float()
        observed_data = self.tokenizer(
            observed_data[:, :, self.cont_list],
            observed_data[:, :, len(self.cont_list) :],
        )
        B, K, L, C = observed_data.shape
        observed_data = observed_data.reshape(B, K, L * C)
        observed_mask = batch["observed_mask"][:, np.newaxis, :]
        observed_mask = observed_mask.to(self.device).float()

        gt_mask = batch["gt_mask"][:, np.newaxis, :]
        gt_mask = gt_mask.to(self.device).float()

        observed_tp = batch["timepoints"].to(self.device).float()

        cut_length = torch.zeros(len(observed_data)).long().to(self.device)
        for_pattern_mask = observed_mask

        graph_features = batch["graph_features"]

        return (
            observed_data,
            observed_mask,
            observed_tp,
            gt_mask,
            for_pattern_mask,
            cut_length,
            graph_features,
        )

Data Preprocessing

In [ ]:
def process_func(path: str, masked_features: list, encode=True, missing_ratio=0):

    pd_data = pd.read_csv(path, header=0)
    data = pd_data.iloc[:, 1:]
    cat_list = []
    for n, i in enumerate(data.columns):
        if len(set(data[i])) <= 5 or type(data[i][0]) == str:
            cat_list.append(n)
    # Swap columns
    temp_list = [i for i in range(data.shape[1]) if i not in cat_list]
    temp_list.extend(cat_list)
    new_cols_order = temp_list

    data = data.reindex(columns=data.columns[new_cols_order])
    column_index = {column: list(data.columns).index(column) for column in data.columns}
    with open("./column_index.pk", "wb") as f:
      pickle.dump(column_index, f)
    print(column_index)
    # create two lists to store position
    cont_list = [i for i in range(0, data.shape[1] - len(cat_list))]
    cat_list = [i for i in range(len(cont_list), data.shape[1])]

    observed_values = data.values
    observed_masks = ~pd.isnull(data)
    observed_masks = observed_masks.values

    # In this section, obtain gt_masks
    masks = observed_masks.copy()
    # for each masked feature, mask the corresponding column of observed values.
    if missing_ratio > 0:
      for col in range(masks.shape[1]):
        obs_indices = np.where(masks[:, col])[0]
        miss_indices = np.random.choice(
          obs_indices, (int)(len(obs_indices) * missing_ratio), replace=False
        )
        masks[miss_indices, col] = False
    for col in masked_features:
        masks[:, column_index[col]] = False
    # gt_mask: 0 for missing elements and manully maksed elements
    gt_masks = masks.reshape(observed_masks.shape)

    # train-test split
    indlist = np.arange(len(observed_values))
    np.random.seed(seed + 1)
    np.random.shuffle(indlist)
    tmp_ratio = 1 / nfold
    start = (int)((nfold - 1) * len(observed_values) * tmp_ratio)
    end = (int)(nfold * len(observed_values) * tmp_ratio)
    test_index = indlist[start:end]
    remain_index = np.delete(indlist, np.arange(start, end))
    np.random.shuffle(remain_index)
    num_train = (int)(len(remain_index) * 1)
    train_index = remain_index[:num_train]
    valid_index = remain_index[num_train:]

    num_cate_list = []
    # set encoder here
    encoder = ce.ordinal.OrdinalEncoder(cols=data.columns[cat_list])
    encoder.fit(data)
    new_df = encoder.transform(data)
    # we now need to transform these masks to the new one, suitable for mixed data types.
    cum_num_bits = 0
    new_observed_masks = observed_masks.copy()
    new_gt_masks = gt_masks.copy()

    for index, col in enumerate(cat_list):
        num_cate_list.append(new_df.iloc[:, col].nunique())
        corresponding_cols = len(
            [
                s
                for s in new_df.columns
                if isinstance(s, str) and s.startswith(str(col) + "_")
            ]
        )
        add_col_num = corresponding_cols
        insert_col_obs = observed_masks[:, col]
        insert_col_gt = gt_masks[:, col]

        for i in range(add_col_num - 1):
            observed_masks = np.insert(
                new_observed_masks, cum_num_bits + col, insert_col_obs, axis=1
            )
            gt_masks = np.insert(
                new_gt_masks, cum_num_bits + col, insert_col_gt, axis=1
            )
        cum_num_bits += add_col_num - 1

    observed_values = new_df.values
    observed_values = np.nan_to_num(observed_values)
    observed_values = observed_values.astype(np.float32)

    processed_data_path_norm = f"./data/nfold-{nfold}-missing_ratio-{missing_ratio}_seed-{seed}_max-min_norm.pk"
    # Here we perform max-min normalization.
    print(
        "--------------Dataset has not been normalized yet. Perform data normalization and store the mean value of each column.--------------"
    )
    # data transformation after train-test split.
    col_num = len(cont_list)
    max_arr = np.zeros(col_num)
    min_arr = np.zeros(col_num)
    mean_arr = np.zeros(col_num)
    for index, k in enumerate(cont_list):
        # Using observed_mask to avoid counting missing values (now represented as 0)
        obs_ind = observed_masks[train_index, k].astype(bool)
        temp = observed_values[train_index, k]
        max_arr[index] = max(temp[obs_ind])
        min_arr[index] = min(temp[obs_ind])

    for index, k in enumerate(cont_list):
        observed_values[:, k] = (
            (observed_values[:, k] - (min_arr[index] - 1))
            / (max_arr[index] - min_arr[index] + 1)
        ) * observed_masks[:, k]

    # Get graph features
    graph_values = observed_values * gt_masks
    graph_features = [{component: np.array([bike[column_index[feature]] for feature in bike_components[component]], dtype=np.float32) for component in bike_components} for bike in graph_values]
    print("graph features", graph_features[0])

    with open("./data/transformed_columns.pk", "wb") as f:
        pickle.dump([cont_list, num_cate_list], f)

    with open("./data/encoder.pk", "wb") as f:
        pickle.dump(encoder, f)

    with open(processed_data_path_norm, "wb") as f:
        pickle.dump(
            [observed_values, observed_masks, gt_masks, graph_features, max_arr, min_arr, train_index, test_index, valid_index], f
        )
    return observed_values, observed_masks, gt_masks, cont_list, graph_features, train_index, test_index, valid_index


class tabular_Dataset(Dataset):
    # eval_length should be equal to attributes number.
    def __init__(self, masked_features, use_index_list=None, seed=0, missing_ratio=0):
        np.random.seed(seed)
        dataset_path = "./new_colored_biked.csv"

        processed_data_path_norm = f"./data/nfold-{nfold}-missing_ratio-{missing_ratio}_seed-{seed}_max-min_norm.pk"
        # self.cont_cols is only saved in .pk file before normalization.
        if not os.path.isfile(processed_data_path_norm):
            (
                self.observed_values,
                self.observed_masks,
                self.gt_masks,
                self.cont_cols,
                self.graph_features,
                self.train_index,
                self.test_index,
                self.valid_index,
            ) = process_func(
                dataset_path,
                masked_features,
                encode=True,
                missing_ratio=missing_ratio,
            )
            print("--------Dataset created--------")

        elif os.path.isfile(processed_data_path_norm):  # load datasetfile
            with open(processed_data_path_norm, "rb") as f:
                self.observed_values, self.observed_masks, self.gt_masks, self.graph_features, _, _, self.train_index, self.test_index, self.valid_index = pickle.load(
                    f
                )
            print("--------Normalized dataset loaded--------")

        if use_index_list is None:
            self.use_index_list = np.arange(len(self.observed_values))
        else:
            self.use_index_list = use_index_list

    def __getitem__(self, org_index):
        index = self.use_index_list[org_index]
        s = {
            "observed_data": self.observed_values[index],
            "observed_mask": self.observed_masks[index],
            "gt_mask": self.gt_masks[index],
            "graph_features": self.graph_features[index],
            "timepoints": np.arange(timepoints),
        }
        return s

    def __len__(self):
        return len(self.use_index_list)


def get_dataloader(masked_features, seed=1, nfold=5, batch_size=16, missing_ratio=0):
    dataset = tabular_Dataset(masked_features, seed=seed, missing_ratio=missing_ratio)
    print(f"Dataset size:{len(dataset)} entries")

    # Now the path exists, so the dataset object initialization performs data loading.
    train_dataset = tabular_Dataset(
        masked_features, use_index_list=dataset.train_index, missing_ratio=missing_ratio, seed=seed
    )
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=1)

    valid_dataset = tabular_Dataset(
        masked_features, use_index_list=dataset.valid_index, missing_ratio=missing_ratio, seed=seed
    )
    valid_loader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=0)

    test_dataset = tabular_Dataset(
        masked_features, use_index_list=dataset.test_index, missing_ratio=missing_ratio, seed=seed
    )
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=0)

    print(f"Training dataset size: {len(train_dataset)}")
    print(f"Validation dataset size: {len(valid_dataset)}")
    print(f"Testing dataset size: {len(test_dataset)}")

    return train_loader, valid_loader, test_loader

Training

In [ ]:
def train(
    model,
    config,
    train_loader,
    valid_loader=None,
    valid_epoch_interval=10,
    foldername="",
  ):
    # Control random seed in the current script.
    torch.manual_seed(0)
    np.random.seed(0)
    optimizer = Adam(model.parameters(), lr=config["lr"], weight_decay=1e-6)
    if foldername != "":
        output_path = foldername + "/model.pth"

    p0 = int(0.25 * config["epochs"])
    p1 = int(0.5 * config["epochs"])
    p2 = int(0.75 * config["epochs"])
    p3 = int(0.9 * config["epochs"])
    lr_scheduler = torch.optim.lr_scheduler.MultiStepLR(
        optimizer, milestones=[p0, p1, p2, p3], gamma=0.5
    )
    # history = {'train_loss':[], 'val_rmse':[]}
    best_valid_loss = 1e10
    model = model.to(device)
    for epoch_no in range(config["epochs"]):
        avg_loss = 0
        model.train()
        with tqdm(train_loader, mininterval=5.0, maxinterval=50.0) as it:
            for batch_no, train_batch in enumerate(it, start=1):
                optimizer.zero_grad()
                # The forward method returns loss.
                loss = model(train_batch)
                loss.backward()
                avg_loss += loss.item()
                optimizer.step()
                it.set_postfix(
                    ordered_dict={
                        "avg_epoch_loss": avg_loss / batch_no,
                        "epoch": epoch_no,
                    },
                    refresh=False,
                )
            lr_scheduler.step()

        if valid_loader is not None and (epoch_no + 1) % valid_epoch_interval == 0:
            if foldername != "":
              torch.save(model, output_path)
            print("Start validation")
            model.eval()
            avg_loss_valid = 0
            # some initial settings
            val_nsample = 15
            val_scaler = 1
            mse_total = 0
            mae_total = 0
            evalpoints_total = 0

            with torch.no_grad():
                with tqdm(valid_loader, mininterval=5.0, maxinterval=50.0) as it:
                    for batch_no, valid_batch in enumerate(it, start=1):
                        output = model.evaluate(valid_batch, val_nsample)
                        # `eval_points` is `target_mask`. `observed_time` is `observed_tp`(10)
                        # `c_target` is `observed_data`
                        (
                            samples,
                            c_target,
                            eval_points,
                            observed_points,
                        ) = output
                        samples = samples.permute(0, 1, 3, 2)[:, :, config["model"]["graph_cond_dim"]:, :]  # (B,nsample,L,K)
                        c_target = c_target.permute(0, 2, 1)[:, config["model"]["graph_cond_dim"]:, :]  # (B,L,K)
                        eval_points = eval_points.permute(0, 2, 1)[:, config["model"]["graph_cond_dim"]:, :]

                        # take the median from samples.
                        samples_median = samples.median(dim=1).values
                        mse_current = (
                            ((samples_median - c_target) * eval_points) ** 2
                        ) * (val_scaler**2)
                        mae_current = (
                            torch.abs((samples_median - c_target) * eval_points)
                        ) * val_scaler

                        mse_total += torch.sum(mse_current, dim=0)
                        evalpoints_total += torch.sum(eval_points, dim=0)

                        it.set_postfix(
                            ordered_dict={
                                "rmse_total": torch.mean(
                                    torch.sqrt(torch.div(mse_total, evalpoints_total))
                                ).item(),
                                "batch_no": batch_no,
                            },
                            refresh=True,
                        )

Evaluating

In [ ]:
def evaluate_ft(
    model, test_loader, nsample=100, scaler=1, mean_scaler=0, foldername=""
  ):

    with open("./data/transformed_columns.pk", "rb") as f:
        cont_list, num_cate_list = pickle.load(f)
    with open("./data/encoder.pk", "rb") as f:
        encoder = pickle.load(f)

    torch.manual_seed(0)
    np.random.seed(0)
    model = model.to(device)
    with torch.no_grad():
        model.eval()
        mse_total = 0
        mae_total = 0
        err_total = np.zeros([len(num_cate_list)])
        err_total_eval_nums = np.zeros([len(num_cate_list)])
        evalpoints_total = 0

        all_target = []
        all_observed_point = []
        all_evalpoint = []
        all_generated_samples = []

        with tqdm(test_loader, mininterval=5.0, maxinterval=50.0) as it:
            for batch_no, test_batch in enumerate(it, start=1):
                output = model.evaluate(test_batch, nsample)
                samples, c_target, eval_points, observed_points = output
                samples = samples.permute(0, 1, 3, 2)  # (B,nsample,L,K)
                c_target = c_target.permute(0, 2, 1)  # (B,L,K)
                eval_points = eval_points.permute(0, 2, 1)
                observed_points = observed_points.permute(0, 2, 1)

                # take the median from samples.
                samples_median = samples.median(dim=1)  # (B, L, K)

                samples_median = model.tokenizer.recover(
                    samples_median.values, len(cont_list)
                )

                c_target = model.tokenizer.recover(c_target, len(cont_list))
                all_target.append(c_target)
                all_evalpoint.append(eval_points)
                all_observed_point.append(observed_points)
                all_generated_samples.append(samples)

                # for continous variables
                mse_current = (
                    (
                        (samples_median[:, cont_list] - c_target[:, cont_list])
                        * eval_points[:, cont_list, 0]
                    )
                    ** 2
                ) * (scaler**2)
                mae_current = (
                    torch.abs(
                        (samples_median[:, cont_list] - c_target[:, cont_list])
                        * eval_points[:, cont_list, 0]
                    )
                ) * scaler

                # for categorical variables
                for i in range(len(num_cate_list)):
                    matched_nums = (
                        samples_median[:, len(cont_list) + i]
                        == c_target[:, len(cont_list) + i]
                        * eval_points[:, len(cont_list) + i, 0]
                    ).sum()
                    eval_nums = eval_points[:, len(cont_list) + i, 0].sum()
                    err_total[i] += eval_nums - matched_nums
                    err_total_eval_nums[i] += eval_nums

                mse_total += torch.sum(mse_current, dim=0)
                mae_total += torch.sum(mae_current, dim=0)
                evalpoints_total += torch.sum(eval_points[:, cont_list, 0], dim=0)
                it.set_postfix(
                    ordered_dict={
                        "rmse_total": torch.mean(
                            torch.sqrt(torch.div(mse_total, evalpoints_total))
                        ).item(),
                        "batch_no": batch_no,
                    },
                    refresh=True,
                )

            with open(foldername + "/result_nsample" + str(nsample) + ".pk", "wb") as f:
                pickle.dump(
                    [
                        torch.mean(
                            torch.sqrt(torch.div(mse_total, evalpoints_total))
                        ).item(),
                        err_total / err_total_eval_nums,
                    ],
                    f,
                )
                print(
                    "RMSE:",
                    torch.mean(
                        torch.sqrt(torch.div(mse_total, evalpoints_total))
                    ).item(),
                )
                print("ERR_CATE:", err_total / err_total_eval_nums)

Main

In [ ]:
train_loader, valid_loader, test_loader = get_dataloader(
    masked_features,
    seed=seed,
    nfold=nfold,
    batch_size=config["train"]["batch_size"],
    missing_ratio=config["model"]["test_missing_ratio"],
)

model = BikeGraphCSDI(config, device)

foldername = "./save/BikeGraphCSDI/"

train(
    model,
    config["train"],
    train_loader,
    valid_loader=valid_loader,
    foldername=foldername,
)

{'BB textfield': 0, 'Seat tube length': 1, 'Stack': 2, 'Head tube upper extension2': 3, 'Seat angle': 4, 'CS textfield': 5, 'FCD textfield': 6, 'Seat tube extension2': 7, 'Head tube lower extension2': 8, 'Head angle': 9, 'Saddle height': 10, 'ERD rear': 11, 'Wheel width rear': 12, 'Dropout spacing': 13, 'SPOKES composite front': 14, 'SPOKES front': 15, 'Wheel diameter front': 16, 'BSD front': 17, 'Wheel diameter rear': 18, 'SPOKES rear': 19, 'Wheel width front': 20, 'SPOKES composite rear': 21, 'ERD front': 22, 'BSD rear': 23, 'SBLADEW front': 24, 'SBLADEW rear': 25, 'FORK0R': 26, 'FORK0L': 27, 'Saddle P': 28, 'Saddle thickness': 29, 'Saddle angle': 30, 'Saddle J': 31, 'Saddle H': 32, 'Saddle E': 33, 'SADDLETIPtoMIDDLE': 34, 'Saddle length': 35, 'Seatpost setback': 36, 'Seatpost LENGTH': 37, 'Lower stack height': 38, 'Upper stack height': 39, 'Headset spacers': 40, 'Stem angle': 41, 'CLAMPOFFSET': 42, 'StemX': 43, 'StemY': 44, 'StemG': 45, 'Collar height': 46, 'Stem length': 47, 'AEROF

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/transformer.py:282: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")
  0%|          | 0/309 [00:00<?, ?it/s]

In [ ]:
evaluate_ft(model, train_loader, nsample=nsample, scaler=1, foldername=foldername)

Qualitative Analysis

In [ ]:
num_sam = 5   # number of samples
num_sim = 150   # number of diffusion simulations
masked_features = ["Stem angle", "Seat tube length", "bottle SEATTUBE0 show", "Handlebar style"]

_, _, test_loader = get_dataloader([], seed=4096, batch_size=num_sam * len(masked_features), nfold=10, missing_ratio=0)
test_index = test_loader.dataset.test_index

model = torch.load("./save/BikeGraphCSDI/model.pth", map_location=device).to(device)
model.device = device
model.graph_embed.device = device

with open("column_index.pk", "rb") as f:
  column_index = pickle.load(f)

test_loader.dataset.gt_masks = np.ones_like(test_loader.dataset.gt_masks)
for n, i in enumerate(masked_features):
  test_loader.dataset.gt_masks[test_loader.dataset.use_index_list[num_sam*n:num_sam*(n+1)], column_index[i]] = False  # Manual mask

sample, _, _, _ = model.evaluate(next(iter(test_loader)), num_sim)
sample = sample.permute(0, 1, 3, 2)

with open("./data/transformed_columns.pk", "rb") as f:
    cont_list, num_cate_list = pickle.load(f)
with open("./data/encoder.pk", "rb") as f:
    encoder = pickle.load(f)

num_feature = len(cont_list) + len(num_cate_list)

temp = torch.zeros(len(masked_features) * num_sam * num_sim, num_feature)
for i in range(len(masked_features) * num_sam):
  for j in range(num_sim):
    temp[i*(num_sim)+j,:] = model.tokenizer.recover(sample[i,j,:,:].unsqueeze(0), len(cont_list))[0]

with open("./data/nfold-10-missing_ratio-0.1_seed-4096_max-min_norm.pk", "rb") as f:
  _, _, _, _, max_arr, min_arr, _, _, _ = pickle.load(f)

for i in cont_list:   # Recover from normalization
  temp[:, i] *= (max_arr[i] - min_arr[i] + 1)
  temp[:, i] += min_arr[i] - 1

data = pd.read_csv("new_colored_biked.csv", header=0)
data = data.iloc[:, 1:]

ref = (pd.read_csv("new_colored_biked_processed.csv", header=0).iloc[:, 1:]).columns
first_cat = 0
for n, i in enumerate(ref):
  if "OHCLASS:" in i:
    first_cat = n
    break

for n, i in enumerate(masked_features):   # Save the local table with truth, masked and generated
  df = pd.DataFrame(np.repeat(data.values[test_index[n*num_sam:(n+1)*num_sam]],num_sim,axis=0))
  df.columns = data.columns
  df.to_csv("truth_{}.csv".format(i))
  df_dum = pd.get_dummies(df, prefix_sep=" OHCLASS: ", columns=df.columns[first_cat:])  # Convert back to one hot encoding
  for c in ref:
    if c not in df_dum.columns:
      df_dum[c] = 0
  df_dum.to_csv("truth_{}_OH.csv".format(i))
  df[i] = pd.NA
  df.to_csv("masked_{}.csv".format(i))

  for c in df_dum.columns:
    if i in c:
      df_dum[c] = pd.NA
  df_dum.to_csv("masked_{}_OH.csv".format(i))
  df = encoder.transform(df)
  df[i] = temp[num_sam * num_sim * (n): num_sam * num_sim * (n+1), column_index[i]]
  df = encoder.inverse_transform(df)
  df.to_csv("generated_{}.csv".format(i))
  df_dum = pd.get_dummies(df, prefix_sep=" OHCLASS: ", columns=df.columns[first_cat:])
  for c in ref:
    if c not in df_dum.columns:
      df_dum[c] = 0
  df_dum.to_csv("generated_{}_OH.csv".format(i))

In [ ]:
def compare_plot(population, feature, who=0, type="cont", num_sim=150):
  truth = pd.read_csv("truth_{}.csv".format(feature)).loc[who*num_sim:(who+1)*num_sim]
  generated = pd.read_csv("generated_{}.csv".format(feature)).loc[who*num_sim:(who+1)*num_sim]
  if type == "cont":
    plt.hist(population[feature], bins=50, alpha=0.5, label='truth', density=True)
    plt.hist(generated[feature], alpha=0.5, label='generated', density=True)
    plt.axvline(x=truth[feature][who*num_sim], ymin=0, ymax=0.1)
    plt.axvline(x=generated[feature].mean() + bias, color="red", ymin=0, ymax=0.1)
    plt.title(feature)
    plt.legend(loc='upper right')
    plt.show()
  else:
    labels = set(population[feature])
    truth = np.array([sum([row[feature]==label for _, row in truth.iterrows()]) for label in labels])
    truth = truth / (truth.sum())
    generated = np.array([sum([row[feature]==label for _, row in generated.iterrows()]) for label in labels])
    generated = generated / (generated.sum())

    x = np.arange(len(labels))  # the label locations
    width = 0.35  # the width of the bars

    fig, ax = plt.subplots()
    rects1 = ax.bar(x - width/2, truth, width, label='truth')
    rects2 = ax.bar(x + width/2, generated, width, label='generated')

    # Add some text for labels, title and custom x-axis tick labels, etc.
    ax.set_title(feature)
    ax.set_xticks(x)
    ax.set_xticklabels(labels)
    ax.legend()

In [ ]:
population = pd.read_csv("new_colored_biked.csv", header=0).iloc[:,1:]
feature = "Stem angle"
compare_plot(population, feature, who=4)

In [ ]:
feature = "Seat tube length"
compare_plot(population, feature, who=4)

In [ ]:
feature = "bottle SEATTUBE0 show"
compare_plot(population, feature, type="cat", who=3)

In [ ]:
feature = "Handbar style"
compare_plot(population, feature, type="cat", who=3)